# $\color{teal}{\text{4.4 Orthogonal Matching Pursuit}}$

This algorithm is a representant of so-called greedy methods. Since the original $\color{green}{(P_{0,\eta})}$ is NP-hard, we use the OMP as an approximation method. It iteratively constructs the support set of the reconstructed sparse vector by extending the current support set with an additional index at each iteration. One drawback of this algorithm is that if a wrong index has been selected, it will remain in all subsequent index sets. The initial support set shall be the empty set and $x^{(0)}$ shall be equal to the all-zero vector. Further, the residual $r^{(0)}$ is chosen to be $y$.

Each iteration of the OMP algorithms covers mainly two steps denoted by `OMP_1` and `OMP_2` and mathematically expressed as

$$ j_{k+1} = \underset{j \in [N]}{\mathrm{argmax}} \{|a^{*}_j \cdot r^{(k)}|\}, \;\;\; S^{(k+1)}=S^{(k)} \cup \{j_{k+1}\}$$
and 
$$ x^{(k+1)} = \underset{z \in \mathbb{C}^{N}}{\mathrm{argmin}} \{||y-Az||_2\} \;\;\; \text{s.t. supp}(z) \subset S^{(k+1)} $$

We further introduce the residual term $r^{k+1} = y - Ax^{(k+1)}$, which can be understood as an error term. The desired output of the algorithm is an $\bar{n}$-sparse vector, where $\bar{n}$ denotes the number of iterations needed for the recovery of $x$. Since `OMP_2` is a least squares problem, we can calculate $x^{(k+1)}$ using the pseudoinverse $A^{\dagger}_{S^{(k+1)}} \in \mathbb{C}^{N \times m}$ that is restricted to the set $S^{(k+1)}$ via

$$ x^{(k+1)}=A^{\dagger}_{S^{(k+1)}}y = (A^{*}_{S^{(k+1)}}A_{S^{(k+1)}})^{-1}A^{*}_{S^{(k+1)}}y.$$

Restricting a matrix on a support set means that that we take all columns indicated in the support set and set the other ones to zero.

In [ ]:
import numpy as np
import random
import math

#omp
m = 200
N = 175
s = 30

#ground truth
x_true = np.zeros(N)
indices = random.sample(range(N),s) #randomly sample s out of 1000 indices
x_true[indices] = np.random.rand(s)

#random measurement matrix
A = np.random.randn(m,N)
A = A/math.sqrt(m)
#simulated measurements
y = A @ x_true

In [ ]:
def get_max_index(residual,matrix,n_clmns):
    matrix_T = matrix.T
    dot_products =[np.dot(matrix_T[i].conjugate(),residual) for i in range(n_clmns)]
    abs_vals = np.absolute(dot_products)
    return np.argmax(abs_vals)

In [ ]:
def restrict_on_support(matrix,support_set):
    """Restrict matrix on support set by copying all elements in the columns indicated by 
    support_set and setting the other ones to zero."""
    
    [rows,clms] = matrix.shape
    matrix_supp = np.zeros((rows,clms), dtype='float32')
    matrix_supp[:,support_set] = matrix[:,support_set]
    return matrix_supp

In [ ]:
def omp(A,y,iters):
    """Implementation of the Orthogonal Matching Pursuit."""   
    from numpy.linalg import pinv
    
    [m,N] = A.shape
    #Initializations
    x_iters = []
    support_set = []
    err_iters = np.zeros(iters)
    x_0 = np.zeros(N)
    r_0 = y
    
    #Iteration
    for i in range(iters):
        if i==0:
            residual = r_0
        print(f"residual: \n {residual[50]}")
        index = get_max_index(residual,A,N)
        print(f"index: {index}")
        support_set.append(index)
        print(support_set)
        
        A_r = restrict_on_support(A,support_set)       
        x_new = np.linalg.pinv(A_r)@y
        
        print(f"x_new: \n {sum(x_new)}")
        residual = y - A@x_new
        x_iters.append(x_new)
        
        err = np.linalg.norm(A@x_new-y,2) #l2-error
        err_iters[i] = err
        print(f"err iteration {i}: {err}")
    
    return x_new, x_iters, err_iters

In [ ]:
n_iters = 100
x_hat, x_iters, err_iters = omp(A,y,n_iters)

In [ ]:
err = np.linalg.norm(x_hat-x_true,2) #l2-error
print(err)